In [8]:
import kagglehub

path = kagglehub.dataset_download("iarunava/cell-images-for-detecting-malaria")

print("Path to dataset files:", path)
image_directory = "/root/.cache/kagglehub/datasets/iarunava/cell-images-for-detecting-malaria/versions/1/cell_images/"



Path to dataset files: /root/.cache/kagglehub/datasets/iarunava/cell-images-for-detecting-malaria/versions/1


In [9]:
print("Parasitized exists:", os.path.exists(image_directory + "Parasitized/"))
print("Uninfected exists:", os.path.exists(image_directory + "Uninfected/"))


Parasitized exists: True
Uninfected exists: True


In [10]:
import kagglehub
import cv2
import os
import numpy as np
from PIL import Image
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [3]:
image_directory= "sell_images/"
SIZE= 64
dataset= []
label=[]


In [11]:
if os.path.exists(image_directory + "Parasitized/"):
    parasitized_images = os.listdir(image_directory + "Parasitized/")

    for i, image_name in enumerate(parasitized_images):
        if image_name.endswith('.png'):
            image = cv2.imread(os.path.join(image_directory, "Parasitized", image_name))
            image = Image.fromarray(image, "RGB")
            image = image.resize((SIZE, SIZE))
            dataset.append(np.array(image))
            label.append(0)

if os.path.exists(image_directory + "Uninfected/"):
    uninfected_images = os.listdir(image_directory + "Uninfected/")

    for i, image_name in enumerate(uninfected_images):
        if image_name.endswith('.png'):
            image = cv2.imread(os.path.join(image_directory, "Uninfected", image_name))
            image = Image.fromarray(image, "RGB")
            image = image.resize((SIZE, SIZE))
            dataset.append(np.array(image))
            label.append(1)

In [12]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(SIZE, SIZE, 3), activation="relu", data_format="channels_last"))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.2))

model.add(Dense(256, activation="relu"))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.2))

model.add(Dense(2, activation="sigmoid"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 31, 31, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 29, 29, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 14, 14, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 512)                 │       3,211,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │             514 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,357,090 (12.81 MB)

 Trainable params: 3,355,426 (12.80 MB)

 Non-trainable params: 1,664 (6.50 KB)

None


In [13]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# Split the data
x_train, x_test, y_train, y_test = train_test_split(dataset, to_categorical(np.array(label)), test_size=0.20, random_state=0)

history = model.fit(
    np.array(x_train),
    y_train,
    batch_size=64,
    verbose=1,
    epochs=5,
    validation_split=0.1,
    shuffle=False

)


print("Test Accuracy: {:.2f}%".format(model.evaluate(np.array(x_test), np.array(y_test))[1] * 100))


Epoch 1/5
311/311 ━━━━━━━━━━━━━━━━━━━━ 124s 382ms/step - accuracy: 0.6436 - loss: 0.7851 - val_accuracy: 0.6286 - val_loss: 1.7900
Epoch 2/5
311/311 ━━━━━━━━━━━━━━━━━━━━ 116s 372ms/step - accuracy: 0.8940 - loss: 0.2713 - val_accuracy: 0.9184 - val_loss: 0.2701
Epoch 3/5
311/311 ━━━━━━━━━━━━━━━━━━━━ 117s 375ms/step - accuracy: 0.9240 - loss: 0.2039 - val_accuracy: 0.9333 - val_loss: 0.1950
Epoch 4/5
311/311 ━━━━━━━━━━━━━━━━━━━━ 119s 382ms/step - accuracy: 0.9332 - loss: 0.1791 - val_accuracy: 0.9302 - val_loss: 0.2042
Epoch 5/5
311/311 ━━━━━━━━━━━━━━━━━━━━ 139s 372ms/step - accuracy: 0.9432 - loss: 0.1572 - val_accuracy: 0.9342 - val_loss: 0.1990
173/173 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - accuracy: 0.9271 - loss: 0.2236
Test Accuracy: 92.47%
